# How to access the preprocessed data?

Speeches of MPs sorted by party in a list. Name of the referee is the first element of the text:

Reden_AfD \
Reden_Gruene\
Reden_CDU\
Reden_FDP\
Reden_Fraktionslos\
Reden_SPD

Call get_remarks() on a speech to extract all remarks that were made during the speech in the following format:

Beifall\
Einwürfe\
Lachen\
Heiterkeit\
thematische_zwischenrufe_spd\
thematische_zwischenrufe_cdu\
thematische_zwischenrufe_fdp\
thematische_zwischenrufe_gruene\
thematische_zwischenrufe_afd\
thematische_zwischenrufe_fraktionslos\
thematische_zwischenrufe_linke\

Beifall, Einwürfe, Lachen and Heiterkeit return dictionaries with the party name as a key and the amount of reactions as the value. Thematische zwischenrufe are contained as strings in a list. 

Call get_missing_mps on the entire meeting content (NOT A SPEECH!) to obtain  lists of the names of missing mps per party and a dictionary with the number of missing mps per party, using the party name as a key and the number of missing mps as a value.

Call get_requests on the entire meeting content (NOT A SPEECH!) to get a list per party containing the mps and the title of a request as well as a dictionary containing the number of requests per party in the plenary meeting



In [177]:
import xml.etree.ElementTree as ET
import re
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
import os

In [178]:
#Read xml Files

filename = '../Data/19026.xml'

tree = ET.parse(filename)

root = tree.getroot()
for child in root:
    print(child.tag, child.attrib)

WAHLPERIODE {}
DOKUMENTART {}
NR {}
DATUM {}
TITEL {}
TEXT {}


In [179]:
#Parse text into one line and clean up linebreaks for easier reading later on

for content in root.iter('TEXT'):
    meeting_content = content.text

meeting_content = " ".join(line.strip() for line in meeting_content.splitlines())
meeting_content = meeting_content.replace("- ","")

In [180]:
party_split = re.split("(\(DIE LINKE\):|\(AfD\):|\(BÜNDNIS 90/DIE GRÜNEN\):|\(CDU/CSU\):|\(FDP\):|\(FRAKTIONSLOS\):|\(SPD\):)", meeting_content)

In [181]:
# Function which returns last words

def lastWords(string):

# split by space and converting
# string to list and
    lis = list(string.split(" "))
    concat = " ".join(lis[-3:])

# returning last 3 elements in list
    return concat

In [182]:
#Splitting text into talking points

Reden_Linke = []
Reden_AfD = []
Reden_Gruene = []
Reden_CDU = []
Reden_FDP = []
Reden_Fraktionslos = []
Reden_SPD = []

for idx, element in enumerate(party_split):

    #Check if text was split on party name

    if element == "(DIE LINKE):":

        #concat last words of previous split element (contains the referees name), actual split element (contains party name) and next split element (contains text of speech)

        Reden_Linke.append(lastWords(party_split[idx-1])+" "+party_split[idx]+" "+party_split[idx+1])

        #Split on name of the moderator to mark the end of the speech and delete none speech text parts

        moderation_split = re.split("Präsident Dr. Wolfgang Schäuble:|Vizepräsident Dr. Hans-Peter Friedrich:", Reden_Linke[-1])
        Reden_Linke[-1] = moderation_split[0]
    if element == "(AfD):":
        Reden_AfD.append(lastWords(party_split[idx-1])+" "+party_split[idx]+" "+party_split[idx+1])
        moderation_split = re.split("Präsident Dr. Wolfgang Schäuble:|Vizepräsident Dr. Hans-Peter Friedrich:", Reden_AfD[-1])
        Reden_AfD[-1] = moderation_split[0]
    if element == "(BÜNDNIS 90/DIE GRÜNEN):":
        Reden_Gruene.append(lastWords(party_split[idx-1])+" "+party_split[idx]+" "+party_split[idx+1])
        moderation_split = re.split("Präsident Dr. Wolfgang Schäuble:|Vizepräsident Dr. Hans-Peter Friedrich:", Reden_Gruene[-1])
        Reden_Gruene[-1] = moderation_split[0]
    if element == "(CDU/CSU):":
        Reden_CDU.append(lastWords(party_split[idx-1])+" "+party_split[idx]+" "+party_split[idx+1])
        moderation_split = re.split("Präsident Dr. Wolfgang Schäuble:|Vizepräsident Dr. Hans-Peter Friedrich:", Reden_CDU[-1])
        Reden_CDU[-1] = moderation_split[0]
    if element == "(FDP):":
        Reden_FDP.append(lastWords(party_split[idx-1])+" "+party_split[idx]+" "+party_split[idx+1])
        moderation_split = re.split("Präsident Dr. Wolfgang Schäuble:|Vizepräsident Dr. Hans-Peter Friedrich:", Reden_FDP[-1])
        Reden_FDP[-1] = moderation_split[0]
    if element == "(FRAKTIONSLOS):":
        Reden_Fraktionslos.append(lastWords(party_split[idx-1])+" "+party_split[idx]+" "+party_split[idx+1])
        moderation_split = re.split("Präsident Dr. Wolfgang Schäuble:|Vizepräsident Dr. Hans-Peter Friedrich:", Reden_Fraktionslos[-1])
        Reden_Fraktionslos[-1] = moderation_split[0]
    if element == "(SPD):":
        Reden_SPD.append(lastWords(party_split[idx-1])+" "+party_split[idx]+" "+party_split[idx+1])
        moderation_split = re.split("Präsident Dr. Wolfgang Schäuble:|Vizepräsident Dr. Hans-Peter Friedrich:", Reden_SPD[-1])
        Reden_SPD[-1] = moderation_split[0]

In [183]:
def get_remarks(string):
    parties = ['LINKE', 'BÜNDNIS 90', 'CDU/CSU', 'SPD', 'FDP', 'AfD', 'fraktionslos']
    einwürfe = []
    beifall = []
    lachen =[]
    heiterkeit = []
    newlist = []
    zuruf = []

    counter_beifall_spd = 0
    counter_beifall_cdu = 0
    counter_beifall_afd = 0
    counter_beifall_fdp = 0
    counter_beifall_linke = 0
    counter_beifall_gruene = 0
    counter_beifall_fraktionslos = 0

    counter_einwürfe_spd = 0
    counter_einwürfe_cdu = 0
    counter_einwürfe_afd = 0
    counter_einwürfe_fdp = 0
    counter_einwürfe_linke = 0
    counter_einwürfe_gruene = 0
    counter_einwürfe_fraktionslos = 0

    counter_lachen_spd = 0
    counter_lachen_cdu = 0
    counter_lachen_afd = 0
    counter_lachen_fdp = 0
    counter_lachen_linke = 0
    counter_lachen_gruene = 0
    counter_lachen_fraktionslos = 0

    counter_heiterkeit_spd = 0
    counter_heiterkeit_cdu = 0
    counter_heiterkeit_afd = 0
    counter_heiterkeit_fdp = 0
    counter_heiterkeit_linke = 0
    counter_heiterkeit_gruene = 0
    counter_heiterkeit_fraktionslos = 0

    thematische_zwischenrufe_spd = []
    thematische_zwischenrufe_cdu = []
    thematische_zwischenrufe_afd = []
    thematische_zwischenrufe_fdp = []
    thematische_zwischenrufe_linke = []
    thematische_zwischenrufe_gruene = []
    thematische_zwischenrufe_fraktionslos = []

    #Find all strings between () brackets and allow for matching over multiple lines

    matches = re.findall( '\((\n*?|[^)]*)\)', string)

    #Convert matches to single line strings and discard elements with fewer than 9 string characters

    for element in matches:
        element = " ".join(line.strip() for line in element.splitlines())
        if len(element) > 9:
            newlist.append(element)

    #Find strings that contain information about parties, all remarks should contain a string indicating party

    for element in newlist:
        if any(party in element for party in parties):
            einwürfe.append(element)

    #Sort renarks into categories. We allow for thematic remarks of specific people and for reactions of entire groups

    for element in einwürfe:
        if "Beifall" in element:
            beifall.append(element)
            #einwürfe.remove(element)
        if "Lachen" in element:
            lachen.append(element)
            #einwürfe.remove(element)
        if "Heiterkeit" in element:
            heiterkeit.append(element)
            #einwürfe.remove(element)
        if "Zuruf" in element:
            zuruf.append(element)
            #einwürfe.remove(element)
        if "[SPD]:" in element:
            thematische_zwischenrufe_spd.append(element)
            #einwürfe.remove(element)
        if "[CDU/CSU]:" in element:
            thematische_zwischenrufe_cdu.append(element)
            #einwürfe.remove(element)
        if "[AfD]:" in element:
            thematische_zwischenrufe_afd.append(element)
            #einwürfe.remove(element)
        if "[FDP]:" in element:
            thematische_zwischenrufe_fdp.append(element)
            #einwürfe.remove(element)
        if "[BÜNDNIS 90/DIE GRÜNEN]:" in element:
            thematische_zwischenrufe_gruene.append(element)
            #einwürfe.remove(element)
        if "[DIE LINKE]:" in element:
            thematische_zwischenrufe_linke.append(element)
            #einwürfe.remove(element)
        if "[FRAKTIONSLOS]:" in element:
            thematische_zwischenrufe_fraktionslos.append(element)
            #einwürfe.remove(element)

    for element in beifall:
        if "SPD" in element:
            counter_beifall_spd += 1
        
        if "CDU/CSU" in element:
            counter_beifall_cdu += 1
        
        if "AfD" in element:
            counter_beifall_afd += 1

        if "FDP" in element:
            counter_beifall_fdp += 1
    
        if "DIE GRÜNEN" in element:
            counter_beifall_gruene += 1

        if "LINKE" in element:
            counter_beifall_linke += 1

        if "FRAKTIONSLOS" in element:
            counter_beifall_fraktionslos += 1

    for element in einwürfe:
        if "SPD" in element:
            counter_einwürfe_spd += 1
        
        if "CDU/CSU" in element:
            counter_einwürfe_cdu += 1
        
        if "AfD" in element:
            counter_einwürfe_afd += 1

        if "FDP" in element:
            counter_einwürfe_fdp += 1
    
        if "DIE GRÜNEN" in element:
            counter_einwürfe_gruene += 1

        if "LINKE" in element:
            counter_einwürfe_linke += 1

        if "FRAKTIONSLOS" in element:
            counter_einwürfe_fraktionslos += 1

    for element in lachen:
        if "SPD" in element:
            counter_lachen_spd += 1
        
        if "CDU/CSU" in element:
            counter_lachen_cdu += 1
        
        if "AfD" in element:
            counter_lachen_afd += 1

        if "FDP" in element:
            counter_lachen_fdp += 1
    
        if "DIE GRÜNEN" in element:
            counter_lachen_gruene += 1

        if "LINKE" in element:
            counter_lachen_linke += 1

        if "FRAKTIONSLOS" in element:
            counter_lachen_fraktionslos += 1

    for element in heiterkeit:
        if "SPD" in element:
            counter_heiterkeit_spd += 1
        
        if "CDU/CSU" in element:
            counter_heiterkeit_cdu += 1
        
        if "AfD" in element:
            counter_heiterkeit_afd += 1

        if "FDP" in element:
            counter_heiterkeit_fdp += 1
    
        if "DIE GRÜNEN" in element:
            counter_heiterkeit_gruene += 1

        if "LINKE" in element:
            counter_heiterkeit_linke += 1

        if "FRAKTIONSLOS" in element:
            counter_heiterkeit_fraktionslos += 1

        

    #print(beifall)
    #print(lachen)
    #print(heiterkeit)
    #print(zuruf)

    #print('Beifall die Linke:' + str(counter_beifall_linke))
    #print('Beifall CDU:' + str(counter_beifall_cdu))
    #print('Beifall SPD:' + str(counter_beifall_spd))
    #print('Beifall AfD:' + str(counter_beifall_afd))
    #print('Beifall Fraktionslos:' + str(counter_beifall_fraktionslos))
    #print('Beifall die Grüne:' + str(counter_beifall_gruene))
    #print('Beifall FDP:' + str(counter_beifall_fdp))


    Beifall = {
        "Linke": counter_beifall_linke,
        "CDU": counter_beifall_cdu,
        "SPD": counter_beifall_spd,
        "AfD": counter_beifall_afd,
        "Fraktionslos": counter_beifall_fraktionslos,
        "Gruene": counter_beifall_gruene,
        "FDP": counter_beifall_fdp,
    }

    #print('Einwürfe die Linke:' + str(counter_einwürfe_linke))
    #print('Einwürfe CDU:' + str(counter_einwürfe_cdu))
    #print('Einwürfe SPD:' + str(counter_einwürfe_spd))
    #print('Einwürfe AfD:' + str(counter_einwürfe_afd))
    #print('Einwürfe Fraktionslos:' + str(counter_einwürfe_fraktionslos))
    #print('Einwürfe die Grüne:' + str(counter_einwürfe_gruene))
    #print('Einwürfe FDP:' + str(counter_einwürfe_fdp))

    Einwürfe = {
        "Linke": counter_einwürfe_linke,
        "CDU": counter_einwürfe_cdu,
        "SPD": counter_einwürfe_spd,
        "AfD": counter_einwürfe_afd,
        "Fraktionslos": counter_einwürfe_fraktionslos,
        "Gruene": counter_einwürfe_gruene,
        "FDP": counter_einwürfe_fdp,
    }

    #print('Lachen die Linke:' + str(counter_lachen_linke))
    #print('Lachen CDU:' + str(counter_lachen_cdu))
    #print('Lachen SPD:' + str(counter_lachen_spd))
    #print('Lachen AfD:' + str(counter_lachen_afd))
    #print('Lachen Fraktionslos:' + str(counter_lachen_fraktionslos))
    #print('Lachen die Grüne:' + str(counter_lachen_gruene))
    #print('Lachen FDP:' + str(counter_lachen_fdp))

    Lachen = {
        "Linke": counter_lachen_linke,
        "CDU": counter_lachen_cdu,
        "SPD": counter_lachen_spd,
        "AfD": counter_lachen_afd,
        "Fraktionslos": counter_lachen_fraktionslos,
        "Gruene": counter_lachen_gruene,
        "FDP": counter_lachen_fdp,
    }

    #print('Heiterkeit die Linke:' + str(counter_heiterkeit_linke))
    #print('Heiterkeit CDU:' + str(counter_heiterkeit_cdu))
    #print('Heiterkeit SPD:' + str(counter_heiterkeit_spd))
    #print('Heiterkeit AfD:' + str(counter_heiterkeit_afd))
    #print('Heiterkeit Fraktionslos:' + str(counter_heiterkeit_fraktionslos))
    #print('Heiterkeit die Grüne:' + str(counter_heiterkeit_gruene))
    #print('Heiterkeit FDP:' + str(counter_heiterkeit_fdp))

    Heiterkeit = {
        "Linke": counter_heiterkeit_linke,
        "CDU": counter_heiterkeit_cdu,
        "SPD": counter_heiterkeit_spd,
        "AfD": counter_heiterkeit_afd,
        "Fraktionslos": counter_heiterkeit_fraktionslos,
        "Gruene": counter_heiterkeit_gruene,
        "FDP": counter_heiterkeit_fdp,
    }


    return(Beifall, 
            Einwürfe, 
            Lachen, 
            Heiterkeit, 
            thematische_zwischenrufe_spd, 
            thematische_zwischenrufe_cdu, 
            thematische_zwischenrufe_fdp, 
            thematische_zwischenrufe_gruene, 
            thematische_zwischenrufe_afd, 
            thematische_zwischenrufe_fraktionslos, 
            thematische_zwischenrufe_linke)





In [190]:
print(get_remarks(Reden_AfD[0]))

({'Linke': 0, 'CDU': 0, 'SPD': 0, 'AfD': 5, 'Fraktionslos': 0, 'Gruene': 0, 'FDP': 0}, {'Linke': 0, 'CDU': 1, 'SPD': 1, 'AfD': 5, 'Fraktionslos': 0, 'Gruene': 5, 'FDP': 0}, {'Linke': 0, 'CDU': 0, 'SPD': 0, 'AfD': 0, 'Fraktionslos': 0, 'Gruene': 0, 'FDP': 0}, {'Linke': 0, 'CDU': 0, 'SPD': 0, 'AfD': 0, 'Fraktionslos': 0, 'Gruene': 0, 'FDP': 0}, [], [], [], ['Katrin Göring-Eckardt [BÜNDNIS 90/DIE GRÜNEN]: Direkt aus dem Berghain! – Weitere Zurufe des Abg . Jürgen Trittin [BÜNDNIS 90/DIE GRÜNEN]', 'Britta Haßelmann [BÜNDNIS 90/DIE GRÜNEN]: Wir haben keines! Im Gegensatz zu Ihnen!', 'Katrin Göring-Eckardt [BÜNDNIS 90/DIE GRÜNEN]: Viel!', 'Britta Haßelmann [BÜNDNIS 90/DIE GRÜNEN]: „Ehrlich“ ist ein gutes Stichwort! Wo sind eigentlich die 27 000 Euro?'], [], [], [])


In [185]:
def get_missing_mps(document):

    missing_mps_DIE_LINKE = {}
    missing_mps_CDUCSU = {}
    missing_mps_FDP = {}
    missing_mps_SPD = {}
    missing_mps_GRUENE = {}
    missing_mps_FRAKTIONSLOS = {}
    missing_mps_AFD = {}

    #create helper arrays

    DIELINKE = []
    CDUCSU= []
    FDP= []
    SPD= []
    GRUENE= []
    FRAKTIONSLOS= []
    AFD= []

     #This split is done to seperate the Anlagen from the rest of the text, it is the only consistently working string we know of
    document = document.replace(" .",".")
    split = document.split("Stenografischen Bericht")

    #This split is done to cut the rest of the file from the missing MPs part of the anlagen. If there is no Anlage 2, we have found that splitting on "satzweiss.com" yields good results too
    

    split = split[1].split("Anlage 2")

    
    missing_mps = split[0].split("Satzweiss.com")
  
    #split into seperate rows to get MP names

    missing_mps = re.split("\d+\.\d+\.\d+",missing_mps[0])

    #assign mps to parties

    for element in missing_mps:

        if "DIE LINKE" in element:
            element = element.split("DIE LINKE")
            DIELINKE.append(element[0])
            #np.append(missing_mps_DIE_LINKE, [[filename],[element[0]]])
            
        if "CDU/CSU" in element:
            element = element.split("CDU/CSU")
            CDUCSU.append(element[0])

        if "FDP" in element:
            element = element.split("FDP")
            FDP.append(element[0])
           

        if "SPD" in element:
            element = element.split("SPD")
            SPD.append(element[0])
        
        if "BÜNDNIS 90/" in element:
            element = element.split("BÜNDNIS 90/")
            GRUENE.append(element[0])
        
        if "fraktionslos" in element:
            element = element.split("fraktionslos")
            FRAKTIONSLOS.append(element[0])

        if "AfD" in element:
            element = element.split("AfD")
            AFD.append(element[0])

    
    #Here the complete names of missing mps per meeting are added to a dictionary with the meeting id in format "19XXX" as a key

    missing_mp_stats = {
        "Linke": len(DIELINKE),
        "CDU": len(CDUCSU),
        "SPD": len(SPD),
        "AfD": len(AFD),
        "Fraktionslos": len(FRAKTIONSLOS),
        "Gruene": len(GRUENE),
        "FDP": len(FDP),
    }



    return DIELINKE, CDUCSU, FDP, SPD, GRUENE, FRAKTIONSLOS, AFD, missing_mp_stats



In [186]:
print(get_missing_mps(meeting_content))

([' * aufgrund gesetzlichen Mutterschutzes Deutscher Bundestag – 19. Wahlperiode – 26. Sitzung. Berlin, Donnerstag, den 19. April 20182472 (A) (C) (B) (D) Namensverzeichnis CDU/CSU Dr. Michael von Abercron Stephan Albani Norbert Maria Altenkamp Philipp Amthor Artur Auernhammer Peter Aumer Dorothee Bär Thomas Bareiß Maik Beermann Manfred Behrens (Börde) Veronika Bellmann Sybille Benning Dr. André Berghegger Melanie Bernstein Christoph Bernstiel Peter Beyer Marc Biadacz Steffen Bilger Peter Bleser Michael Brand (Fulda) Dr. Reinhard Brandl Dr. Ralf Brauksiepe Dr. Helge Braun Silvia Breher Sebastian Brehm Heike Brehmer Ralph Brinkhaus Dr. Carsten Brodesser Gitta Connemann Astrid Damerow Alexander Dobrindt Marie-Luise Dött Michael Donth Hansjörg Durz Thomas Erndl Hermann Färber Uwe Feiler Enak Ferlemann Axel E. Fischer (KarlsruheLand) Dr. Maria Flachsbarth Thorsten Frei Dr. Hans-Peter Friedrich (Hof) Michael Frieser Hans-Joachim Fuchtel Ingo Gädechens Dr. Thomas Gebhart Alois Gerig Eberhard

In [187]:
def get_requests(string):

    DIELINKE = []
    CDUCSU= []
    FDP= []
    SPD= []
    GRUENE= []
    FRAKTIONSLOS= []
    AFD= []

    requests = []

    results = re.findall( 'Antrag der Abgeordneten............................................................................................................................................................................................................................................................................................................................................................', string)

    for element in results:
        element = element.split('Drucksache')
        requests.append(element[0])

    for element in requests:

        if "DIE LINKE" in element:
            DIELINKE.append(element)
            #np.append(missing_mps_DIE_LINKE, [[filename],[element[0]]])
            
        if "CDU/CSU" in element:
            CDUCSU.append(element)

        if "FDP" in element:
            FDP.append(element)
           

        if "SPD" in element:
            SPD.append(element)
        
        if "BÜNDNIS 90/" in element:
            GRUENE.append(element)
        
        if "fraktionslos" in element:
            FRAKTIONSLOS.append(element)

        if "AfD" in element:
            AFD.append(element)

        

    requests_stats = {
        "Linke": len(DIELINKE),
        "CDU": len(CDUCSU),
        "SPD": len(SPD),
        "AfD": len(AFD),
        "Fraktionslos": len(FRAKTIONSLOS),
        "Gruene": len(GRUENE),
        "FDP": len(FDP),
    }


    return DIELINKE, CDUCSU, FDP, SPD, GRUENE, FRAKTIONSLOS, AFD, requests_stats

In [188]:
print(get_requests(meeting_content))

(['Antrag der Abgeordneten Dr . Alexander S . Neu, Heike Hänsel, Sevim Dağdelen, weiterer Abgeordneter und der Fraktion DIE LINKE: Keine Stationierung neuer Nuklearwaffen in der Bundesrepublik ‒ INF-Vertrag erhalten ', 'Antrag der Abgeordneten Dr . Kirsten Tackmann, Kersten Steinke, Dr . Gesine Lötzsch, weiterer Abgeordneter und der Fraktion DIE LINKE sowie der Abgeordneten Friedrich Ostendorff, Harald Ebner, Renate Künast, weiterer Abgeordneter und der Fraktion BÜNDNIS 90/DIE GRÜNEN: Weidetierprämie für Schafe und Ziegen jetzt auf den Weg bringen ', 'Antrag der Abgeordneten Susanne  Ferschl, Sabine Zimmermann (Zwickau), Katja Kipping, weiterer Abgeordneter und der Fraktion DIE LINKE: Armut in Deutschland den Kampf ansagen ', 'Antrag der Abgeordneten Ingrid Remmers, Amira Mohamed Ali, Jörg Cezanne, weiterer Abgeordneter und der Fraktion DIE LINKE: Hersteller zur wirksamen technischen Nachrüstung von Diesel-Pkw auf ihre Kosten verpflichten – Fahrverbote vermeiden ', 'Antrag der Abgeordn